This notebook started from John Bart's notebook of example code showing how to read the audio clip data using the librosa.

I changed this to read the "one_week" recording. 

In [1]:
# remove warnings
import warnings
warnings.filterwarnings('ignore')
# ---

%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

import pandas as pd
pd.options.display.max_columns = 100

import numpy as np

import librosa
import librosa.display

# dir containing sound files
srcdir = './one_week'

Read clip paths from recording folder¶
Parses date/time for each clip
Sorts clips by time

In [2]:
import os
import fnmatch
from datetime import datetime,timedelta  

def read_clipnames(srcdir):
    """Walk through base folder and collect paths for all sound files.
        parse date and time info, sort, and return as a dataframe"""
    
    clipexts=['*.mp3']
    datefmt='%Y-%m-%d--%H.%M.mp3'

    # search through source folder for sound files
    # save clip path and date (parsed from filename)
    clippath = []
    clipdate = []
    for ext in clipexts:
        for root, dirnames, filenames in os.walk(srcdir):
            for filename in fnmatch.filter(filenames, ext):
                clippath.append(os.path.join(root, filename).replace('\\','/'))
                clipdate.append(datetime.strptime(filename, datefmt))
                
    # get sort index
    idx = np.argsort(clipdate)
    # retun sorted dataframe
    return pd.DataFrame({'date': np.array(clipdate)[idx], 
                       'path':  np.array(clippath)[idx]})  
df = read_clipnames(srcdir)

Read audio data and plot spectrographs and waveforms of several clips¶


In [3]:
numclips = 3
hoplength=None
fftsize=256
numsamps=100000

# read and plot some sound clips
for i, (dt, path) in df.iloc[:numclips].iterrows():
    # convert mp3 to audio samples
    y, sr = librosa.load(path, sr=None)
    y = y[:numsamps]

    # abs date-time of each sample (disable - takes to long)
    #x = [dt+timedelta(seconds=t) for t in np.arange(0,len(y))/sr]

    # time of each sample (seconds rel to clip start)
    x = np.arange(0,len(y))/sr
    
    # calc spectrograph
    D = librosa.stft(np.array(y), hop_length=hoplength, n_fft=fftsize)
    
    # Pre-compute a global reference power from the input spectrum
    rp = np.max(np.abs(D))
    
    plt.figure(figsize=(15,10))
    plt.suptitle('clip %d, time %s'%(i, dt.strftime('%y-%m-%d %H:%M:00')))

    # display spectrograph
    plt.subplot(3,2,1)
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(D),ref=rp), 
                             sr=sr, y_axis='linear',cmap='coolwarm')
    
    # plot waveform
    plt.subplot(3,2,2)
    plt.plot(x, y, 'b-', linewidth=1)
    plt.xlabel('time (s)')
    plt.ylabel('amplitude');
    
    # plot chromagram
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hoplength, n_mfcc=13)
    plt.subplot(3,2,3)
    chromagram = librosa.feature.chroma_stft(y, sr=sr, hop_length=hoplength)
    chgramplot = librosa.display.specshow(chromagram, hop_length=hoplength,cmap='coolwarm')
    
    plt.subplot(3,2,4)
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    librosa.display.specshow(D, y_axis='log')
    
    plt.subplot(3,2,5)
    librosa.display.waveplot(y[:10000])
    plt.ylim(-1, 1)
    

NoBackendError: 